In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import tensorflow
from keras.preprocessing import image
from keras.models import Sequential,Model
from keras.layers import Input,Lambda,Dense,Flatten
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
import cv2 as cv2

In [2]:
from platform import python_version
python_version()

'3.9.7'

In [3]:
import tensorflow as tf
print(tf.__version__)

2.8.0


In [4]:
import os
import cv2
data=[]
categories=['with_mask','without_mask']

for category in categories:
    path=os.path.join(r'D:\Sandesh\Data Science\Deep Learning\Mask_Detection\train',category)
    label=categories.index(category)
    
    for file in os.listdir(path):
        img_path=os.path.join(path,file)
        img=cv2.imread(img_path)
        img=cv2.resize(img,(224,224))
        data.append([img,label])
        

In [5]:
len(data)

1585

In [6]:
import random
random.shuffle(data)

In [7]:
x=[]
y=[]

for features,label in data:
    x.append(features)
    y.append(label)

In [8]:
X=np.array(x)
Y=np.array(y)
print(len(X),len(Y))
print(X.shape)
print(Y.shape)


1585 1585
(1585, 224, 224, 3)
(1585,)


In [9]:
Y

array([0, 1, 1, ..., 0, 1, 1])

In [10]:
X=X/255

In [11]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.20)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((1268, 224, 224, 3), (317, 224, 224, 3), (1268,), (317,))

In [12]:
vgg=VGG19()

In [13]:
vgg.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [14]:
model=Sequential()

for layer in vgg.layers[:-1]:
    model.add(layer)

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [16]:
for layers in model.layers:
    layers.trainable=False

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [18]:
model.add(Dense(units=1,activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [19]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics='accuracy')


In [ ]:
model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Epoch 1/5
 5/40 [==>...........................] - ETA: 4:48 - loss: 0.7243 - accuracy: 0.5312

In [ ]:
import cv2 
cap = cv2.VideoCapture(0)

while True:
    
    ret,frame=cap.read()
    
    # Call the Detection Method
    
    img=cv2.resize(frame,(224,224))
   
    y_pred=detect_face_mask(img)
    
    coods=detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))
    
    for x,y,w,h in coods:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    
    y_pred=(np.round(y_pred).astype(int))
    
    if y_pred == 0:
        draw_label(frame,"Mask",(30,30),(0,255,0))
    else:
        draw_label(frame,"No_Mask",(30,30),(0,0,255))
    
    cv2.imshow("window",frame)
    
    if cv2.waitKey(1) & 0XFF ==ord('a'):
        break
    
cv2.destroyAllWindows()


In [ ]:
def detect_face_mask(img):
    
    y_pred=model.predict(img.reshape(1,224,224,3))
    return y_pred[0][0]
    

In [ ]:
def draw_label(img,text,pos,bg_color):

    text_size=cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    
    end_x=pos[0]+text_size[0][0]+2
    end_y=pos[1]+text_size[0][1]+2
    
    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)
    
    

In [ ]:
# with mask = 0 , without_mask = 1

sample1=cv2.imread(r'D:/Sandesh/Resume/Profile Img.png')

sample1=cv2.resize(sample1,(224,224))

y_output=detect_face_mask(sample1)
y_output = np.round(y_output).astype(int)
y_output

In [ ]:
# with mask = 0 , without_mask = 1

sample2=cv2.imread(r'D:\Sandesh\Data Science\Deep Learning\Mask_Detection\train\with_mask\pra2.jpg')
sample2=cv2.resize(sample2,(224,224))
y_output=detect_face_mask(sample2)
y_output = np.round(y_output).astype(int)
y_output

In [ ]:

cap = cv2.VideoCapture(0)

while True:
    
    ret,frame=cap.read()
    coods=detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))
    
    for x,y,w,h in coods:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3)
          
    cv2.imshow("Windows",frame)        
    

    if cv2.waitKey(1) & 0XFF ==ord('a'):
        break
    
cv2.destroyAllWindows()



In [ ]:
har_cascade=cv2.CascadeClassifier('D:\Sandesh\Data Science\Deep Learning\Mask_Detection\haarcascade_frontalface_default.xml')

In [ ]:

def detect_face(img):
    
    coords=har_cascade.detectMultiScale(img)
    return coords
    